# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = "aapl" 
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'MEG(LT A/TASSB)LRLD GNAS OEQANKEC',
 'calculationPrice': 'close',
 'open': 0,
 'openTime': None,
 'openSource': 'faocilif',
 'close': 0,
 'closeTime': None,
 'closeSource': 'ilfoicfa',
 'high': 0,
 'highTime': None,
 'highSource': None,
 'low': 0,
 'lowTime': None,
 'lowSource': None,
 'latestPrice': 139.9,
 'latestSource': 'Close',
 'latestTime': 'June 29, 2021',
 'latestUpdate': 1693739750396,
 'latestVolume': None,
 'iexRealtimePrice': 139.27,
 'iexRealtimeSize': 21,
 'iexLastUpdated': 1628864756264,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 139.38,
 'previousVolume': 64764323,
 'change': 1.58,
 'changePercent': 0.012,
 'volume': None,
 'iexMarketPercent': 216.64845774971346,
 'iexVolume': 2214917,
 'avgTotalVolume': 7596

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

31.7

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [12]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ],
            index = my_columns
        ),
            ignore_index = True
        )
#         print(symbol)

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [13]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [14]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,676.54,4.98,N/A
1,CINF,117.28,6.22,N/A
2,ZION,55.20,6.42,N/A
3,AFL,56.11,7.02,N/A
4,UNM,28.86,7.67,N/A
5,HRB,24.07,7.92,N/A
6,CE,151.83,8.62,N/A
7,COO,410.85,8.64,N/A
8,KIM,21.00,8.92,N/A
9,CSX,31.68,8.96,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [16]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [18]:
portfolio_input()

Enter the value of your portfolio:one million dollars
That's not a number! 
 Try again:
Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [23]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,BIO,676.54,4.98,29
1,CINF,117.28,6.22,170
2,ZION,55.20,6.42,362
3,AFL,56.11,7.02,356
4,UNM,28.86,7.67,693
5,HRB,24.07,7.92,830
6,CE,151.83,8.62,131
7,COO,410.85,8.64,48
8,KIM,21.00,8.92,952
9,CSX,31.68,8.96,631


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [38]:
symbol = "AAPL"
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# print(data[symbol]['advanced-stats']['enterpriseValue'])
# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebita = enterprise_value/ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


17.427536144812606


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [48]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_dataframe

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(","):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_ebita = enterprise_value/ebitda
        except TypeError:
            np.NaN
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A', 
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebita,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
                ],
                index = rv_columns
            ),
            ignore_index = True
        )

In [49]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,150.51,N/A,51.78,N/A,9.34,N/A,7.73,N/A,31.975447,N/A,15.290244,N/A,N/A
1,AAL,21.91,N/A,-1.44,N/A,-1.79,N/A,1.1,N/A,-3.702078,N/A,3.051066,N/A,N/A
2,AAP,209.80,N/A,22.56,N/A,3.88,N/A,1.27,N/A,11.592143,N/A,2.925527,N/A,N/A
3,AAPL,141.49,N/A,31.2,N/A,33.59,N/A,6.94,N/A,22.440751,N/A,17.489686,N/A,N/A
4,ABBV,114.50,N/A,38.48,N/A,15.04,N/A,4.06,N/A,13.001089,N/A,8.532232,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.57,N/A,32,N/A,-4.47,N/A,6.09,N/A,23.922645,N/A,10.336998,N/A,N/A
501,ZBH,163.79,N/A,59.32,N/A,2.82,N/A,4.88,N/A,29.348362,N/A,8.615445,N/A,N/A
502,ZBRA,549.78,N/A,46.16,N/A,11.8,N/A,6.18,N/A,33.463703,N/A,13.634534,N/A,N/A
503,ZION,55.10,N/A,6.47,N/A,1.2,N/A,2.62,N/A,5.215221,N/A,2.572878,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [52]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,248.60,N/A,None,N/A,None,N/A,None,N/A,15.762611,N/A,NaN,N/A,N/A
71,BRK.B,284.76,N/A,None,N/A,None,N/A,None,N/A,20.211089,N/A,NaN,N/A,N/A
118,CTL,11.00,N/A,9.94,N/A,None,N/A,None,N/A,25.906250,N/A,NaN,N/A,N/A
136,DISCK,29.46,N/A,None,N/A,None,N/A,None,N/A,7.710209,N/A,NaN,N/A,N/A
165,ETFC,51.16,N/A,14.39,N/A,None,N/A,None,N/A,17.941806,N/A,NaN,N/A,N/A
168,EVRG,62.76,N/A,None,N/A,None,N/A,None,N/A,12.733990,N/A,NaN,N/A,N/A
186,FLIR,58.72,N/A,33.14,N/A,None,N/A,None,N/A,10.273903,N/A,NaN,N/A,N/A
190,FOX,36.49,N/A,None,N/A,None,N/A,None,N/A,15.079440,N/A,NaN,N/A,N/A
192,FRC,187.60,N/A,None,N/A,None,N/A,None,N/A,7.425605,N/A,NaN,N/A,N/A
204,GOOG,2544.67,N/A,None,N/A,None,N/A,None,N/A,3.292295,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [53]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [54]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [65]:
from scipy.stats import percentileofscore as score
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,150.51,N/A,51.78,0.829703,9.34,0.788119,7.73,0.79703,31.975447,0.863366,15.290244,0.827723,N/A
1,AAL,21.91,N/A,-1.44,0.120792,-1.79,0.083168,1.10,0.105941,-3.702078,0.037624,3.051066,0.134653,N/A
2,AAP,209.80,N/A,22.56,0.417822,3.88,0.519802,1.27,0.141584,11.592143,0.29505,2.925527,0.124752,N/A
3,AAPL,141.49,N/A,31.20,0.574257,33.59,0.958416,6.94,0.766337,22.440751,0.689109,17.489686,0.869307,N/A
4,ABBV,114.50,N/A,38.48,0.69505,15.04,0.871287,4.06,0.544554,13.001089,0.380198,8.532232,0.50297,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.57,N/A,32.00,0.583168,-4.47,0.081188,6.09,0.724752,23.922645,0.734653,10.336998,0.645545,N/A
501,ZBH,163.79,N/A,59.32,0.859406,2.82,0.39703,4.88,0.621782,29.348362,0.833663,8.615445,0.508911,N/A
502,ZBRA,549.78,N/A,46.16,0.8,11.80,0.831683,6.18,0.726733,33.463703,0.883168,13.634534,0.770297,N/A
503,ZION,55.10,N/A,6.47,0.130693,1.20,0.135644,2.62,0.372277,5.215221,0.057426,2.572878,0.10297,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [70]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,150.51,N/A,51.78,0.829703,9.34,0.788119,7.73,0.79703,31.975447,0.863366,15.290244,0.827723,0.821188
1,AAL,21.91,N/A,-1.44,0.120792,-1.79,0.083168,1.10,0.105941,-3.702078,0.037624,3.051066,0.134653,0.096436
2,AAP,209.80,N/A,22.56,0.417822,3.88,0.519802,1.27,0.141584,11.592143,0.29505,2.925527,0.124752,0.299802
3,AAPL,141.49,N/A,31.20,0.574257,33.59,0.958416,6.94,0.766337,22.440751,0.689109,17.489686,0.869307,0.771485
4,ABBV,114.50,N/A,38.48,0.69505,15.04,0.871287,4.06,0.544554,13.001089,0.380198,8.532232,0.50297,0.598812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.57,N/A,32.00,0.583168,-4.47,0.081188,6.09,0.724752,23.922645,0.734653,10.336998,0.645545,0.553861
501,ZBH,163.79,N/A,59.32,0.859406,2.82,0.39703,4.88,0.621782,29.348362,0.833663,8.615445,0.508911,0.644158
502,ZBRA,549.78,N/A,46.16,0.8,11.80,0.831683,6.18,0.726733,33.463703,0.883168,13.634534,0.770297,0.802376
503,ZION,55.10,N/A,6.47,0.130693,1.20,0.135644,2.62,0.372277,5.215221,0.057426,2.572878,0.10297,0.159802


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [ ]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [72]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

<ipython-input-72-cfd3313cd451>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values(by = 'RV Score', inplace = True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.47,N/A,7.67,0.136634,0.5630,0.087129,0.4367,0.021782,3.584703,0.049505,0.432332,0.009901,0.06099
1,MCK,200.50,N/A,-6.96,0.10198,-1505.1500,0.00396,0.1318,0.009901,7.692860,0.110891,2.580994,0.10495,0.066337
2,AIV,6.87,N/A,-74.64,0.025743,2.0000,0.291089,-31.5400,0.00198,-44.180035,0.007921,-52.391855,0.005941,0.066535
3,L,54.51,N/A,-413.77,0.00396,0.8611,0.09901,1.0000,0.093069,7.853834,0.124752,0.987497,0.027723,0.069703
4,AIG,47.50,N/A,-10.81,0.089109,0.6916,0.091089,0.9836,0.085149,5.893591,0.065347,0.945538,0.025743,0.071287
5,HFC,32.78,N/A,-35.00,0.049505,1.0500,0.112871,0.4767,0.025743,8.465503,0.150495,2.789250,0.110891,0.089901
6,AAL,21.91,N/A,-1.44,0.120792,-1.7900,0.083168,1.1000,0.105941,-3.702078,0.037624,3.051066,0.134653,0.096436
7,ABC,118.43,N/A,-6.28,0.106931,-86.6400,0.009901,0.1232,0.007921,8.563833,0.154455,4.154967,0.207921,0.097426
8,ALL,129.72,N/A,11.52,0.188119,1.5800,0.193069,0.8324,0.073267,2.514750,0.041584,0.825919,0.019802,0.103168
9,BA,240.73,N/A,-11.77,0.083168,-7.9600,0.037624,2.5600,0.368317,-19.146521,0.019802,-42.027806,0.007921,0.103366


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [76]:
portfolio_input()

Enter the value of your portfolio:2500000


In [79]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,28.47,1756,7.67,0.136634,0.5630,0.087129,0.4367,0.021782,3.584703,0.049505,0.432332,0.009901,0.06099
1,MCK,200.50,249,-6.96,0.10198,-1505.1500,0.00396,0.1318,0.009901,7.692860,0.110891,2.580994,0.10495,0.066337
2,AIV,6.87,7278,-74.64,0.025743,2.0000,0.291089,-31.5400,0.00198,-44.180035,0.007921,-52.391855,0.005941,0.066535
3,L,54.51,917,-413.77,0.00396,0.8611,0.09901,1.0000,0.093069,7.853834,0.124752,0.987497,0.027723,0.069703
4,AIG,47.50,1052,-10.81,0.089109,0.6916,0.091089,0.9836,0.085149,5.893591,0.065347,0.945538,0.025743,0.071287
5,HFC,32.78,1525,-35.00,0.049505,1.0500,0.112871,0.4767,0.025743,8.465503,0.150495,2.789250,0.110891,0.089901
6,AAL,21.91,2282,-1.44,0.120792,-1.7900,0.083168,1.1000,0.105941,-3.702078,0.037624,3.051066,0.134653,0.096436
7,ABC,118.43,422,-6.28,0.106931,-86.6400,0.009901,0.1232,0.007921,8.563833,0.154455,4.154967,0.207921,0.097426
8,ALL,129.72,385,11.52,0.188119,1.5800,0.193069,0.8324,0.073267,2.514750,0.041584,0.825919,0.019802,0.103168
9,BA,240.73,207,-11.77,0.083168,-7.9600,0.037624,2.5600,0.368317,-19.146521,0.019802,-42.027806,0.007921,0.103366


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [80]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [81]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [82]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [83]:
writer.save()